In [1]:
import os
# Turn off GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


In [2]:
import tensorflow as tf
from glob import glob
import os

from utils.cityscapes import CitySequence
from models import generate_model, generate_model_2
from modelclass import ModelGen

In [3]:
print(tf.config.list_physical_devices('GPU'))
print(tf.__version__)

[]
2.7.1


2022-04-20 15:24:07.455352: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-04-20 15:24:07.455377: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: super-computer
2022-04-20 15:24:07.455381: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: super-computer
2022-04-20 15:24:07.455437: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 510.54.0
2022-04-20 15:24:07.455450: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 510.54.0
2022-04-20 15:24:07.455454: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 510.54.0


In [4]:
GENERAL_PATH = os.path.join("model-saved", "model-1", "city")
MODEL_NAME = "city_dice_fine_model-1.hdf5"
MODEL_PATH = os.path.join(GENERAL_PATH, MODEL_NAME)

image_size = 256
batch_size = 16
num_classes = 30
lr = 0.00001
alpha = 1.

##### Dataset

In [5]:
DATA_DIR = "../cityscapes"
DATA_TYPE = "gtFine"

TYPE = "train"
x_train_dir = sorted(glob(os.path.join(DATA_DIR, DATA_TYPE, "leftImg8bit", TYPE, "*/*_leftImg8bit.png")))
y_train_dir = sorted(glob(os.path.join(DATA_DIR, DATA_TYPE, "gtFine", TYPE, "*/*_gtFine_labelTrainIds.png")))

TYPE = "val"
x_val_dir = sorted(glob(os.path.join(DATA_DIR, DATA_TYPE, "leftImg8bit", TYPE, "*/*_leftImg8bit.png")))
y_val_dir = sorted(glob(os.path.join(DATA_DIR, DATA_TYPE, "gtFine", TYPE, "*/*_gtFine_labelTrainIds.png")))

x_train = CitySequence(x_dir=x_train_dir, y_dir=y_train_dir, batch_size=batch_size, image_size=image_size, horizontal_flip=True, brightness=[True, 0.1])
x_val = CitySequence(x_dir=x_val_dir, y_dir=y_val_dir, batch_size=batch_size, image_size=image_size)



##### Model


In [6]:
model = ModelGen(image_size=image_size, num_classes=num_classes, alpha=alpha, generate_model=generate_model, path=MODEL_PATH)
model.summary()

2022-04-20 15:24:07.826612: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 128, 128, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 128, 128, 32  128         ['Conv1[0][0]']                  
                                )                                                             

##### Compile - Train


In [ ]:
model.compile()

##### Compile - Train

In [ ]:
model.train(x_train=x_train, x_val=x_val, batch_size=batch_size, epochs=15, save_path=MODEL_PATH, monitor="val_loss")

#### **Quantization**


##### TFLITE Converter


In [7]:
from random import shuffle
from glob import glob
import os
import sys
import importlib

def representative_dataset_gen():
    DATA_DIR = "../cityscapes/gtFine"
    IMAGE_PATH = os.path.join(DATA_DIR, "leftImg8bit/train/*/*_leftImg8bit.png")
    images = glob(IMAGE_PATH)
    shuffle(images)
    images = images[0:75]

    for img in images:
        aux = tf.io.read_file(img)
        aux = tf.image.decode_image(aux, channels=3)
        aux.set_shape([None, None, 3])
        aux = tf.image.resize(images=aux,
                              size=[256, 256]) / 127.5 - 1
        yield [tf.reshape(aux, (1, aux.shape[0], aux.shape[1], aux.shape[2]))]
        
model.toTFlite(representative_dataset_gen=representative_dataset_gen)

0 of 3 : SELECT_TF_OPS


2022-04-20 15:24:13.763764: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /tmp/tmp3_3by4kd/assets
Estimated count of arithmetic ops: 0  ops, equivalently 0  MACs


2022-04-20 15:24:32.001429: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:363] Ignored output_format.
2022-04-20 15:24:32.001453: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:366] Ignored drop_control_dependency.
2022-04-20 15:24:32.002052: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmp3_3by4kd
2022-04-20 15:24:32.032570: I tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2022-04-20 15:24:32.032608: I tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: /tmp/tmp3_3by4kd
2022-04-20 15:24:32.159430: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-04-20 15:24:32.548600: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/tmp3_3by4kd
2022-04-20 15:24:32.729389: I tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serve }; Status: success: OK. Took 727337

Estimated count of arithmetic ops: 0  ops, equivalently 0  MACs


...

INFO:tensorflow:Assets written to: /tmp/tmpipc7yy0t/assets
2022-04-20 15:25:08.185894: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:363] Ignored output_format.
2022-04-20 15:25:08.185920: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:366] Ignored drop_control_dependency.
2022-04-20 15:25:08.186106: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpipc7yy0t
2022-04-20 15:25:08.216942: I tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2022-04-20 15:25:08.216973: I tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: /tmp/tmpipc7yy0t
2022-04-20 15:25:08.347196: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-04-20 15:25:08.728851: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/tmpipc7yy0t
2022-04-20 15:25:08.917326: I tensorflow/cc/saved_model/loader.cc:301] SavedMode

Modelo Salvo em model-saved/model-1/city/TFLITE_BUILTINS_city_dice_fine_model-1.tflite
2 of 3 : TFLITE_BUILTINS_INT8
INFO:tensorflow:Assets written to: /tmp/tmpldpu832h/assets


INFO:tensorflow:Assets written to: /tmp/tmpldpu832h/assets
2022-04-20 15:25:52.320510: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:363] Ignored output_format.
2022-04-20 15:25:52.320537: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:366] Ignored drop_control_dependency.
2022-04-20 15:25:52.320723: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpldpu832h
2022-04-20 15:25:52.352130: I tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2022-04-20 15:25:52.352160: I tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: /tmp/tmpldpu832h
2022-04-20 15:25:52.485579: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-04-20 15:25:52.879689: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/tmpldpu832h
2022-04-20 15:25:53.074005: I tensorflow/cc/saved_model/loader.cc:301] SavedMode

Modelo Salvo em model-saved/model-1/city/TFLITE_BUILTINS_INT8_city_dice_fine_model-1.tflite


fully_quantize: 0, inference_type: 6, input_inference_type: 0, output_inference_type: 0
